In [ ]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# Extract settings from config.yaml
pretrained_model = "bert-base-uncased"
num_classes = 2  # Adjust based on your task (e.g., binary classification)
max_len = 128

# Define the GPU device
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(pretrained_model)

# Load the model architecture
model = AutoModelForSequenceClassification.from_pretrained(pretrained_model, num_labels=num_classes)

# Load trained weights
model_path = "output/model/2024-11-25/20-12-01/checkpoint-44147/model.safetensors"
state_dict = torch.load(model_path, map_location=device)  # Load weights onto GPU directly
model.load_state_dict(state_dict)

# Move the model to the GPU
model.to(device)
model.eval()

print("Model is ready on", device)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipykernel_3473738/947141877.py:20: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We

FileNotFoundError: [Errno 2] No such file or directory: '/output/model/2024-11-25/checkpoint-49050/model.safetensors'

In [ ]:
# Example input data
input_texts = ["This is a test input for the model.", "Another example sentence."]

# Tokenize the inputs
inputs = tokenizer(
    input_texts,
    return_tensors="pt",
    truncation=True,
    padding=True,
    max_length=max_len,
)

# Move inputs to the same device as the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
inputs = {key: val.to(device) for key, val in inputs.items()}


In [ ]:
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)

print("Predictions:", predictions.cpu().numpy())


In [ ]:
output_file = "/output/model/2024-11-25/prediction.txt"

with open(output_file, "w") as f:
    for idx, pred in enumerate(predictions.cpu().numpy()):
        f.write(f"{idx + 1} {pred}\n")

print(f"Predictions saved to {output_file}")
